# 01 — ASEC Ingestion (CPS Annual Social and Economic Supplement)

Unique cleaning and variable construction for ASEC. Output must conform to the **analysis-ready schema** (same as AHS) so that `02_analysis_ASEC.ipynb` and `03_comparative_master.ipynb` can use it with `scripts/core_metrics.py`.

Save output to `data/processed/asec_analysis_ready.csv` with **identical column naming** as in `ANALYSIS_READY_SCHEMA`.

In [ ]:
import os
import sys
import pandas as pd

REPO_ROOT = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()
sys.path.insert(0, os.path.join(REPO_ROOT, "scripts"))
from core_metrics import ANALYSIS_READY_SCHEMA

DATA_RAW = os.path.join(REPO_ROOT, "data", "raw", "asec")
DATA_PROCESSED = os.path.join(REPO_ROOT, "data", "processed")
os.makedirs(DATA_PROCESSED, exist_ok=True)
print("Target:", ANALYSIS_READY_SCHEMA["target_col"], "| Features:", len(ANALYSIS_READY_SCHEMA["feature_cols"]))

## ASEC-specific cleaning

1. Place ASEC raw files in `data/raw/asec/`.
2. Map ASEC variables to the same schema (e.g. multigenerational → `Multigen_Rate`, income → `Median_HH_Income` or equivalent).
3. Write to `data/processed/asec_analysis_ready.csv` with standardized column names.

In [ ]:
# After building df_asec:
# out_path = os.path.join(DATA_PROCESSED, "asec_analysis_ready.csv")
# df_asec.to_csv(out_path, index=False)
# print("Saved:", out_path)